In [2]:
import platform

In [3]:
print(platform.architecture())

('64bit', 'WindowsPE')


In [4]:
#여러가지 형태소 분석기가 있다
from konlpy.tag import Hannanum #형태소 나눠주는 것
from konlpy.tag import Okt #각 형태소 별로 분류(Tagging 작업)
hannanum = Hannanum()
print(hannanum.morphs(u'대한민국 제 21대 국회의원 선거는 2020년.'))

['대한민국', '저', '의', '21대', '국회의원', '서', 'ㄴ', '것', '이', '는', '2020년', '.']


In [5]:
if __name__=='__main__':
    tagger=Okt()
    part_of_speech = tagger.pos('[영동군청]코로나19 확진 159명(확진자접촉 56, 증상발현 101, 무증상 2) ▲검사 실시.')
    print(part_of_speech) #튜플 형태로 출력
    noun_list = tagger.nouns('[영동군청]코로나19 확진 159명(확진자접촉 56, 증상발현 101, 무증상 2) ▲검사 실시.')
    print(noun_list) #리스트 형식으로 출력
    

[('[', 'Punctuation'), ('영동', 'Noun'), ('군청', 'Noun'), (']', 'Punctuation'), ('코로나', 'Noun'), ('19', 'Number'), ('확진', 'Noun'), ('159', 'Number'), ('명', 'Noun'), ('(', 'Punctuation'), ('확', 'Noun'), ('진자', 'Noun'), ('접촉', 'Noun'), ('56', 'Number'), (',', 'Punctuation'), ('증상', 'Noun'), ('발현', 'Noun'), ('101', 'Number'), (',', 'Punctuation'), ('무증상', 'Noun'), ('2', 'Number'), (')', 'Punctuation'), ('▲', 'Foreign'), ('검사', 'Noun'), ('실시', 'Noun'), ('.', 'Punctuation')]
['영동', '군청', '코로나', '확진', '명', '확', '진자', '접촉', '증상', '발현', '무증상', '검사', '실시']


In [13]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
# if __name__=='__main__':
#     #원하는 열 이름을 title_list에 저장해둠
#     t_file_name = open('C:/Users/seon/Title_List.txt','r',encoding='utf-8')
#     title_list=[]
#     for line in t_file_name.readlines():
#         title_list.append(line[:-1])
#     t_file_name.close()
    
#     #csv파일 불러오기
#     dataset = pd.read_csv('C:/Users/seon/data.csv')
    
#     tagger = Okt()
#     for title in title_list:
#         #DTM 표현
#         cv = CountVectorizer()
#         corpus  = []
        
#         for doc_num in range(250653):
#             noun_list = tagger.nouns(dataset[title].loc[doc_num])
#             corpus.append(''.join(noun_list))
#     DTM_Array = cv.fit_transform(corpus).toarray()
#     feature_names = cv.get_feature_names()
#     DTM_DataFrame = pd.DataFrame(DTM_Array,columns=feature_names)
#     DTM_DataFrame.to_csv('C:/Users/seon/DTM.csv',encoding='utf-8-sig')
            

In [21]:
import numpy as np
import pandas as pd
import io
import re

In [52]:
data = pd.read_csv('C:/Users/seon/data2.csv')
data.head()

create_date  location_id location_name  md101_sn  \
0  2022/03/08 13:54:33           84      경상북도 성주군    146496   
1  2022/03/08 13:43:49           85      경상북도 안동시    146495   
2  2022/03/08 13:17:44          195      전라남도 완도군    146494   
3  2022/03/08 13:10:43          212      전라북도 임실군    146493   
4  2022/03/08 13:08:00          214      전라북도 전주시    146492   

                                                 msg send_platform  
0  [성주군] ▲3.8.(화) 코로나19 확진자 67명(관내56,타지역11) 발생▲마스...           cbs  
1  [안동시청] 관내 강한 바람으로 인해 대형 산불 발생 위험이 매우 높습니다. 산림 ...           cbs  
2  [완도군청] 목욕탕, 사우나 등 다중이용시설 확진자 급속확산 중 ▶KF마스크 착용,...           cbs  
3  [임실군청] 3.8.(화) 13시 기준 확진자 135명 발생 / 개인 방역수칙 준수...           cbs  
4  [전주시청]3.8(화) 현재기준 2,731명 발생(전일 2,577명) O방역수칙 준...           cbs

In [53]:
cbs = data[['msg']]
cbs.head()

msg
0  [성주군] ▲3.8.(화) 코로나19 확진자 67명(관내56,타지역11) 발생▲마스...
1  [안동시청] 관내 강한 바람으로 인해 대형 산불 발생 위험이 매우 높습니다. 산림 ...
2  [완도군청] 목욕탕, 사우나 등 다중이용시설 확진자 급속확산 중 ▶KF마스크 착용,...
3  [임실군청] 3.8.(화) 13시 기준 확진자 135명 발생 / 개인 방역수칙 준수...
4  [전주시청]3.8(화) 현재기준 2,731명 발생(전일 2,577명) O방역수칙 준...

In [54]:
cbs.loc[0,'msg']

'[성주군] ▲3.8.(화) 코로나19 확진자 67명(관내56,타지역11) 발생▲마스크착용 철저▲타지역방문 및 모임 자제▲유증상시 즉시 검사'

In [47]:
k=0

for i in cbs['msg']:
    text = re.compile('[ㄱ-ㅎ|\d\ㅏ-ㅣ|가-힣]+').findall(i)
    cbs.loc[k,'msg'] = ' '.join(text).strip()
    k+=1

In [48]:
cbs.loc[1,'msg']

'안동시청관내강한바람으로인해대형산불발생위험이매우높습니다산림인접지소각금지재처리주의입산시흡연금지등산불예방에협조바랍니다'

In [56]:
cbs.head()

msg  \
0  [성주군] ▲3.8.(화) 코로나19 확진자 67명(관내56,타지역11) 발생▲마스...   
1  [안동시청] 관내 강한 바람으로 인해 대형 산불 발생 위험이 매우 높습니다. 산림 ...   
2  [완도군청] 목욕탕, 사우나 등 다중이용시설 확진자 급속확산 중 ▶KF마스크 착용,...   
3  [임실군청] 3.8.(화) 13시 기준 확진자 135명 발생 / 개인 방역수칙 준수...   
4  [전주시청]3.8(화) 현재기준 2,731명 발생(전일 2,577명) O방역수칙 준...   

                                           msg_token  
0  [[, 성주군, ], ▲, 3.8, .(, 화, ), 코로나, 19, 확, 진자, ...  
1  [[, 안동시청, ], 관내, 강한, 바람, 으로, 인해, 대형, 산불, 발생, 위...  
2  [[, 완도군청, ], 목욕탕, ,, 사우나, 등, 다중, 이용, 시설, 확, 진자...  
3  [[, 임실군, 청, ], 3.8, .(, 화, ), 13시, 기준, 확, 진자, ...  
4  [[, 전주, 시청, ], 3.8, (, 화, ), 현재, 기준, 2,731, 명,...

In [57]:
def tokenizer(row):
    return tagger.morphs(row)

In [58]:
cbs['msg_token'] = cbs['msg'].apply(tokenizer)
cbs.head()

C:\Users\seon\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


msg  \
0  [성주군] ▲3.8.(화) 코로나19 확진자 67명(관내56,타지역11) 발생▲마스...   
1  [안동시청] 관내 강한 바람으로 인해 대형 산불 발생 위험이 매우 높습니다. 산림 ...   
2  [완도군청] 목욕탕, 사우나 등 다중이용시설 확진자 급속확산 중 ▶KF마스크 착용,...   
3  [임실군청] 3.8.(화) 13시 기준 확진자 135명 발생 / 개인 방역수칙 준수...   
4  [전주시청]3.8(화) 현재기준 2,731명 발생(전일 2,577명) O방역수칙 준...   

                                           msg_token  
0  [[, 성주군, ], ▲, 3.8, .(, 화, ), 코로나, 19, 확, 진자, ...  
1  [[, 안동시청, ], 관내, 강한, 바람, 으로, 인해, 대형, 산불, 발생, 위...  
2  [[, 완도군청, ], 목욕탕, ,, 사우나, 등, 다중, 이용, 시설, 확, 진자...  
3  [[, 임실군, 청, ], 3.8, .(, 화, ), 13시, 기준, 확, 진자, ...  
4  [[, 전주, 시청, ], 3.8, (, 화, ), 현재, 기준, 2,731, 명,...

In [59]:
def tokenizer(row):
    return tagger.nouns(row)
cbs['msg_token'] = cbs['msg'].apply(tokenizer)
cbs.head()
    

C:\Users\seon\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


msg  \
0  [성주군] ▲3.8.(화) 코로나19 확진자 67명(관내56,타지역11) 발생▲마스...   
1  [안동시청] 관내 강한 바람으로 인해 대형 산불 발생 위험이 매우 높습니다. 산림 ...   
2  [완도군청] 목욕탕, 사우나 등 다중이용시설 확진자 급속확산 중 ▶KF마스크 착용,...   
3  [임실군청] 3.8.(화) 13시 기준 확진자 135명 발생 / 개인 방역수칙 준수...   
4  [전주시청]3.8(화) 현재기준 2,731명 발생(전일 2,577명) O방역수칙 준...   

                                           msg_token  
0  [성주군, 화, 코로나, 확, 진자, 명, 관내, 지역, 발생, 마스크, 착용, 철...  
1  [안동시청, 관내, 바람, 대형, 산불, 발생, 위험, 매우, 산림, 인접, 소각,...  
2  [완도군청, 목욕탕, 사우나, 등, 다중, 이용, 시설, 확, 진자, 급속, 확산,...  
3  [임실군, 청, 화, 기준, 확, 진자, 명, 발생, 개인, 방역, 칙, 준수, 지...  
4  [전주, 시청, 화, 현재, 기준, 명, 발생, 전일, 명, 방역, 칙, 준수, 의...